[Origin](https://raw.githubusercontent.com/JetBrains/lets-plot-kotlin/master/docs/examples/jupyter-notebooks/distributions.ipynb)

In [1]:
%useLatestDescriptors
%use lets-plot
import java.util.Random
%use ggdsl(0.1.4-dev-13)

In [2]:
val rand = java.util.Random(123)
val n = 200
val data = mapOf<String, List<Any>>(
    "cond" to List(n) { "A" } + List(n) { "B" },
    "rating" to List(n) { rand.nextGaussian() } + List(n) { rand.nextGaussian() * 1.5 + 1.5 },
)

In [3]:
val cond = source<String>("cond")
val rating = "rating"<Double>()

In [4]:
// Basic histogram of "rating"
val p = letsPlot(data) { x = "rating" } + ggsize(500, 250)
p + geomHistogram(binWidth=0.5)

In [5]:
plot(data) {
    histogram(rating, bins = Bins.byWidth(0.5)) {}
    
    layout {
        size = 500 to 250
    }
}

In [6]:
// Histogram overlaid with kernel density curve
//  - histogram with density instead of count on y-axis
//  - overlay with transparent density plot

p + geomHistogram(binWidth=0.5, color="black", fill="white") { y = "..density.." } +
    geomDensity(alpha=0.2, fill=0xFF6666)

In [7]:
plot(data) {
    histogram(rating, bins = Bins.byWidth(0.5)) {
        y(Stat.DENSITY)
        borderLineColor(Color.BLACK)
        fillColor(Color.WHITE)
    }
    
    density(rating) {
        alpha(0.2)
        fillColor(Color.fromHex("#ff6666"))
    }
    
    layout {
        size = 500 to 250
    }
}

In [8]:
p + geomHistogram(binWidth=.5, color="black", fill="white") +
    geomVLine(xintercept=(data["rating"] as List<Double>).average(), color="red", linetype="dashed", size=1.0)

In [9]:
plot(data) {
    
    histogram(rating, bins = Bins.byWidth(.5)) {
        borderLineColor(Color.BLACK)
        fillColor(Color.WHITE)
    }
    
    
    vLine { 
        x((data["rating"] as List<Double>).average())
        color(Color.RED)
        type(LineType.DASHED)
        width(1.0)
    }
    
    layout {
        size = 500 to 250
    }
}

### Histogram and density plots with multiple groups

In [10]:
val p1 = letsPlot(data) {x = "rating"; fill="cond"} + ggsize(500, 250)

// Default histogram (stacked)
p1 + geomHistogram(binWidth=0.5)

In [11]:
plot(data) {
    histogram(rating, bins = Bins.byWidth(0.5)) {
        fillColor(cond)
    }
    layout {
        size = 500 to 250
    }
}

In [12]:
// Overlaid histograms
p1 + geomHistogram(binWidth=0.5, alpha=0.7, position=Pos.identity)

In [13]:
plot(data) {
    histogram(rating, bins = Bins.byWidth(0.5)) {
        alpha(0.7)
        fillColor(cond)
        position = Position.Identity
    }
    layout {
        size = 500 to 250
    }
}

In [14]:
// Interleaved histograms
p1 + geomHistogram(binWidth=0.5, position=Pos.dodge)

In [15]:
plot(data) {
    histogram(rating, bins = Bins.byWidth(0.5)) {
        fillColor(cond)
        position = Position.Dodge()
    }
    layout {
        size = 500 to 250
    }
}

In [16]:
// Density plot
val p2 = ggplot(data) {x="rating"; color="cond"} + ggsize(500, 250)
p2 + geomDensity()

In [17]:
plot(data) {
    density(rating) {
        borderLineColor(cond)
    }
    layout {
        size = 500 to 250
    }
}

In [18]:
// Density plot with semi-transparent fill
p2 + geomDensity(alpha=.3) {fill="cond"} 

In [19]:
plot(data) {
    density(rating) {
        alpha(.3)
        borderLineColor(cond)
        fillColor(cond)
    }
    layout {
        size = 500 to 250
    }
}

In [20]:
// Find the mean of each group
val means = (data["cond"] as List<String> zip data["rating"] as List<Double>)
        .groupBy(keySelector = { it.first }, valueTransform = { it.second })
        .mapValues { it.value.average() }
val cdat: NamedData = mapOf(
    "cond" to means.keys.toList(),
    "rating" to means.values.toList()
)
cdat

{cond=[A, B], rating=[-0.011843241476365302, 1.5547269440141214]}

In [21]:
// Overlaid histograms with means
p2 + geomHistogram(alpha=.3, position=Pos.identity, size=0.0, bins=10) {fill="cond"} +
     geomVLine(data=cdat, linetype="dashed", size=1.0) {xintercept="rating"; color="cond"}


In [22]:
plot(data) {
    histogram(rating, bins = Bins.byNumber(10)) {
        alpha(.3)
        position = Position.Identity
        borderLineWidth(0.0)
        fillColor(cond)
    }
    vLine(){
        data = cdat.toMutableMap()
        x(rating)
        color(cond)
        type(LineType.DASHED)
        width(1.0)
    }
    layout {
        size = 500 to 250
    }
}

In [23]:
// Use frqpoly instead of histogram
p2 + geomFreqpoly(bins=10) {color="cond"} +
     geomVLine(data=cdat, linetype="dashed", size=1.0) {xintercept="rating"; color="cond"}


In [24]:
plot(data) {
    freqPoly(rating, bins = Bins.byNumber(10)) {
        lineColor(cond)
    }
    vLine(){
        data = cdat.toMutableMap()
        x(rating)
        color(cond)
        type(LineType.DASHED)
        width(1.0)
    }
    layout {
        size = 500 to 250
    }
}

In [25]:
// Density plots with means
p2 + geomDensity() +
     geomVLine(data=cdat, linetype="dashed", size=1.0) {xintercept="rating"; color="cond"}

In [26]:
plot(data) {
    density(rating) {
        borderLineColor(cond)
    }
    vLine(){
        data = cdat.toMutableMap()
        x(rating)
        color(cond)
        type(LineType.DASHED)
        width(1.0)
    }
    layout {
        size = 500 to 250
    }
}

### Using facets

In [27]:
ggplot(data) {x="rating"} + 
    geomHistogram(binWidth=.5, color="black", fill="white") +
    facetGrid("cond")

In [28]:
plot(data) {
    histogram(rating, bins = Bins.byWidth(.5)) {
        borderLineColor(Color.BLACK)
        fillColor(Color.WHITE)
    }
    
    facetGridX(cond)
}

### Box plots

In [29]:
// A basic box plot
val p3 = ggplot(data) {x="cond"; y="rating"} + ggsize(300, 200)
p3 + geomBoxplot()

In [30]:
plot(data) {
    boxplot(cond, rating) {
        
    }
    layout {
        size = 300 to 200
    }
}

In [31]:
// A basic box with the conditions colored
p3 + geomBoxplot {fill="cond"}

In [32]:
plot(data) {
    boxplot(cond, rating) {
        fillColor(cond)
    }
    layout {
        size = 300 to 200
    }
}

In [33]:
// Style outliers
p3 + geomBoxplot(outlierColor="red", outlierShape=8, outlierSize=5)

In [34]:
plot(data) {
    boxplot(cond, rating) {
        outlier {
            color(Color.RED)
            symbol(Symbol.ASTERIX)
            size(5.0)
        }
    }
    layout {
        size = 300 to 200
    }
}